In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#importing nltk libraries
import nltk
#nltk.download('all')
from nltk.corpus import stopwords
import re
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.corpus import wordnet
from nltk import FreqDist
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.sparse import hstack
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV 
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score,precision_score, confusion_matrix, accuracy_score,classification_report

from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB, GaussianNB,BernoulliNB
from sklearn.linear_model import LogisticRegression

from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier

import warnings
warnings.filterwarnings('ignore')

C:\Users\katiy\anaconda3\envs\tensorflow_gpu\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
C:\Users\katiy\anaconda3\envs\tensorflow_gpu\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [2]:
df = pd.read_csv('flipkart_review.csv')
df.head()

,Unnamed: 0,Review_title,Reiew_text,Ratings
0,0,Simply awesome,Superb design value for money,5
1,1,Best in the market!,"Excellent, fast, Efficient, If you are looking...",5
2,2,Wonderful,Love one,5
3,3,Great product,Nice product and same as seen as the product.t...,5
4,4,Pretty good,As of now okay doing good performance for the ...,4


In [3]:
df = df.drop(columns='Unnamed: 0', axis=0)

In [4]:
df.shape

(3228, 3)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3228 entries, 0 to 3227
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Review_title  3228 non-null   object
 1   Reiew_text    3228 non-null   object
 2   Ratings       3228 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 75.8+ KB


In [6]:
df.isna().sum()

Review_title    0
Reiew_text      0
Ratings         0
dtype: int64

In [7]:
df.isnull().sum()

Review_title    0
Reiew_text      0
Ratings         0
dtype: int64

**There is no null or na values in the dataframe**

In [8]:
df.head()

,Review_title,Reiew_text,Ratings
0,Simply awesome,Superb design value for money,5
1,Best in the market!,"Excellent, fast, Efficient, If you are looking...",5
2,Wonderful,Love one,5
3,Great product,Nice product and same as seen as the product.t...,5
4,Pretty good,As of now okay doing good performance for the ...,4


In [9]:
df['Review'] =  df.Review_title+ ' ' + df.Reiew_text 

In [10]:
df.head()

,Review_title,Reiew_text,Ratings,Review
0,Simply awesome,Superb design value for money,5,Simply awesome Superb design value for money
1,Best in the market!,"Excellent, fast, Efficient, If you are looking...",5,"Best in the market! Excellent, fast, Efficient..."
2,Wonderful,Love one,5,Wonderful Love one
3,Great product,Nice product and same as seen as the product.t...,5,Great product Nice product and same as seen as...
4,Pretty good,As of now okay doing good performance for the ...,4,Pretty good As of now okay doing good performa...


In [11]:
df.drop('Reiew_text', inplace=True, axis = 1)

In [12]:
df.drop('Review_title', inplace=True, axis = 1)

In [13]:
df.Review[100]

'Really Nice Cons.\n1.Lite or Dim display colour in 1080p resolution while Online Video Streaming.\n2.Battery Backup is only 3 hours after normal uses but in description shows 9.5 hours.\n\nPros\n1.If you want to buy this Laptop for college or office purpose then buy.\n2.Best in this price segment🏆\n3.Window boot in 5 sec\n4. 8/512⚡️\n5.lite gaming and System Sound Game recording in 60fps.\n6.mircrosoft office student 2021 pre- installed(lifetime validity).'

## Lets Get rid of abbreviations

In [14]:
import contractions

In [15]:
df['Review'] = df.Review.apply(lambda x: contractions.fix(x))

In [16]:
df.Review[100]

'Really Nice Cons.\n1.Lite or Dim display colour in 1080p resolution while Online Video Streaming.\n2.Battery Backup is only 3 hours after normal uses but in description shows 9.5 hours.\n\nPros\n1.If you want to buy this Laptop for college or office purpose then buy.\n2.Best in this price segment🏆\n3.Window boot in 5 sec\n4. 8/512⚡️\n5.lite gaming and System Sound Game recording in 60fps.\n6.mircrosoft office student 2021 pre- installed(lifetime validity).'

In [17]:
df.Ratings.value_counts()

5    2143
4     721
3     180
1     156
2      28
Name: Ratings, dtype: int64

**Above data shows that data distribution is uneven that might create biasing in training or may be in the data split.**
**Some category might miss**

In [18]:
from imblearn.over_sampling import RandomOverSampler

In [19]:
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
corpus = []
for i in range(0, len(df)):
  review = re.sub('[^a-zA-Z]', ' ', df['Review'][i])
  review = review.lower()
  review = review.split()
  ps = PorterStemmer()
  all_stopwords = stopwords.words('english')
  all_stopwords.remove('not')
  review = [ps.stem(word) for word in review if not word in set(all_stopwords)]
  review = ' '.join(review)
  corpus.append(review)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\katiy\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
corpus[100]

'realli nice con lite dim display colour p resolut onlin video stream batteri backup hour normal use descript show hour pro want buy laptop colleg offic purpos buy best price segment window boot sec lite game system sound game record fp mircrosoft offic student pre instal lifetim valid'

In [21]:
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
cv = CountVectorizer(max_features = 5000)
X = cv.fit_transform(corpus).toarray()
y = df.iloc[:,0].values

In [22]:
over = RandomOverSampler()
X, y = over.fit_resample(X,y)

In [23]:
print('1: ',np.count_nonzero(y == 1)), print('2: ',np.count_nonzero(y==2)), print('3: ',np.count_nonzero(y==3)),
print('4: ', np.count_nonzero(y==4)), print('5: ', np.count_nonzero(y==5))

1:  2143
2:  2143
3:  2143
4:  2143
5:  2143


(None, None)

In [24]:
len(y)

10715

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

In [26]:
X_test.shape, X_train.shape

((2143, 3043), (8572, 3043))

In [27]:
from sklearn.naive_bayes import GaussianNB
classifier = GaussianNB()
classifier.fit(X_train, y_train)

GaussianNB()

In [28]:
y_pred = classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

[[1 1]
 [4 4]
 [4 4]
 ...
 [4 5]
 [3 3]
 [2 4]]


In [29]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[411   0   0   0   0]
 [  0 437   0   0   0]
 [  5   8 438   0   0]
 [  6   4  49 370   2]
 [  0   1  25 134 253]]


0.8908072795146991

In [30]:
from sklearn.neighbors import KNeighborsClassifier
classifier_KNN = KNeighborsClassifier(n_neighbors = 5, metric = 'minkowski', p = 2)
classifier_KNN.fit(X_train, y_train)

KNeighborsClassifier()

In [31]:
y_pred = classifier_KNN.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[411   0   0   0   0]
 [  0 437   0   0   0]
 [  0   0 451   0   0]
 [  3   0  15 399  14]
 [ 12   2  16  62 321]]
0.9421371908539431
              precision    recall  f1-score   support

           1       0.96      1.00      0.98       411
           2       1.00      1.00      1.00       437
           3       0.94      1.00      0.97       451
           4       0.87      0.93      0.89       431
           5       0.96      0.78      0.86       413

    accuracy                           0.94      2143
   macro avg       0.94      0.94      0.94      2143
weighted avg       0.94      0.94      0.94      2143



In [32]:
from sklearn.ensemble import RandomForestClassifier
classifier_RF = RandomForestClassifier(n_estimators = 20, criterion = 'entropy', random_state = 0)
classifier_RF.fit(X_train, y_train)

RandomForestClassifier(criterion='entropy', n_estimators=20, random_state=0)

In [33]:
y_pred = classifier_RF.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[411   0   0   0   0]
 [  0 437   0   0   0]
 [  0   0 451   0   0]
 [  0   0   0 428   3]
 [  2   0   0   9 402]]
0.9934671021931871
              precision    recall  f1-score   support

           1       1.00      1.00      1.00       411
           2       1.00      1.00      1.00       437
           3       1.00      1.00      1.00       451
           4       0.98      0.99      0.99       431
           5       0.99      0.97      0.98       413

    accuracy                           0.99      2143
   macro avg       0.99      0.99      0.99      2143
weighted avg       0.99      0.99      0.99      2143



In [34]:
## Line by line comparision of prdicted value vs actual value
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1)[1:15])

[[4 4]
 [4 4]
 [2 2]
 [2 2]
 [3 3]
 [3 3]
 [1 1]
 [1 1]
 [4 4]
 [5 5]
 [5 5]
 [2 2]
 [5 5]
 [2 2]]


### Try Some other method for accuracy improvemnet

In [35]:
X = df.Review
y = df.Ratings

In [36]:
##  Unwanted text removal
def scrub_words(text):
    #removes html markup
    text = re.sub("(<.*?>)", "", text)
    #remove non-ascii and digits
    text = re.sub("(\\W)", " ", text)
    text = re.sub("(\\d)", "", text)
    #remove white space
    text = text.strip()
    return text

### Text cleaning 

In [60]:
def scrub_words(text):
    ## removing hyperlinks in the text
    text = re.sub("(<.*?>)", "", text)

    ## removing non-sci characters
    text = re.sub("(\\W)", " ", text)
    text = re.sub("(\\d)", "", text)

    #remove white space
    text = text.strip()
    return text

In [70]:
 X = df.Review.apply(lambda x: scrub_words(x)) ## clean text 

In [71]:
X[100]

'Really Nice Cons   Lite or Dim display colour in p resolution while Online Video Streaming   Battery Backup is only  hours after normal uses but in description shows   hours   Pros  If you want to buy this Laptop for college or office purpose then buy   Best in this price segment   Window boot in  sec        lite gaming and System Sound Game recording in fps   mircrosoft office student  pre  installed lifetime validity'

In [72]:
word_vectorizer = TfidfVectorizer(token_pattern=r'\w{1,}',
                             strip_accents='unicode',
                              stop_words= 'english',
                              ngram_range= (2,6),
                              max_features=100000)

In [73]:
train_word_features = word_vectorizer.fit_transform(X)

In [74]:
train_word_features.shape

(3228, 100000)

In [75]:
char_vectorizer = TfidfVectorizer(
                                sublinear_tf = True,
                                strip_accents = 'unicode',
                                analyzer = 'char',
                                stop_words = 'english',
                                ngram_range = (2,6),
                                max_features = 50000)

train_char_features = char_vectorizer.fit_transform(X)

In [43]:
## concatenating both values
train_features = hstack([train_word_features, train_char_features])

In [76]:
train_features

<3228x150000 sparse matrix of type '<class 'numpy.float64'>'
	with 2756527 stored elements in COOrdinate format>

In [77]:
# splitting the data 
X_train, X_test, y_train, y_test = train_test_split(train_features, y, test_size = 0.25, random_state = 0, )

In [79]:
# lets check the shapes of the all the splits 
print("x_train", X_train.shape)
print("x_test", X_test.shape)
print("y_train", y_train.shape)
print("y_test", y_test.shape)

x_train (2421, 150000)
x_test (807, 150000)
y_train (2421,)
y_test (807,)


In [80]:
## Various classification methods
rf = RandomForestClassifier()
svc = LinearSVC()
lr = LogisticRegression(solver='lbfgs')
mnb = MultinomialNB()
xgb = XGBClassifier(verbosity=0)
bnb = BernoulliNB()
sgd = SGDClassifier()

In [81]:
### Creating a function to train the models
def BuiltModel(model):
    print('*'*30+model.__class__.__name__+'*'*30)
    model.fit(X_train,y_train)
    pred = model.predict(X_test)

    accuracy = accuracy_score(y_test,pred)*100

    print(f"Accuracy Score:", accuracy)
    print("---------------------------------------------------")

    #confusion matrix & classification report
    
    print(f"CLASSIFICATION REPORT : \n {classification_report(y_test,pred)}")
    print(f"Confusion Matrix : \n {confusion_matrix(y_test,pred)}\n")

In [49]:
### Training all the models
for model in [lr,svc,bnb,mnb,sgd,rf,xgb]:
    BuiltModel(model)

******************************LogisticRegression******************************
Accuracy Score: 90.83023543990086
---------------------------------------------------
CLASSIFICATION REPORT : 
               precision    recall  f1-score   support

           1       1.00      0.71      0.83        41
           2       1.00      0.17      0.29         6
           3       1.00      0.52      0.68        48
           4       0.92      0.81      0.86       164
           5       0.90      0.99      0.94       548

    accuracy                           0.91       807
   macro avg       0.96      0.64      0.72       807
weighted avg       0.91      0.91      0.90       807

Confusion Matrix : 
 [[ 29   0   0   0  12]
 [  0   1   0   2   3]
 [  0   0  25   7  16]
 [  0   0   0 133  31]
 [  0   0   0   3 545]]

******************************LinearSVC******************************
Accuracy Score: 92.56505576208178
---------------------------------------------------
CLASSIFICATION REPORT : 
 

In [83]:
### Training all the models
for model in [lr,svc,bnb,mnb,sgd,rf,xgb]:
    BuiltModel(model)

******************************LogisticRegression******************************
Accuracy Score: 90.83023543990086
---------------------------------------------------
CLASSIFICATION REPORT : 
               precision    recall  f1-score   support

           1       1.00      0.71      0.83        41
           2       1.00      0.17      0.29         6
           3       1.00      0.52      0.68        48
           4       0.92      0.81      0.86       164
           5       0.90      0.99      0.94       548

    accuracy                           0.91       807
   macro avg       0.96      0.64      0.72       807
weighted avg       0.91      0.91      0.90       807

Confusion Matrix : 
 [[ 29   0   0   0  12]
 [  0   1   0   2   3]
 [  0   0  25   7  16]
 [  0   0   0 133  31]
 [  0   0   0   3 545]]

******************************LinearSVC******************************
Accuracy Score: 92.56505576208178
---------------------------------------------------
CLASSIFICATION REPORT : 
 

In [52]:
#lets selects different parameters for tuning
grid_params = {
                'penalty':['l2'],
                'loss':['hinge','squared_hinge'],
                'multi_class': ['ovr'],
                'intercept_scaling':[2,3],
                'dual':[True,False],
                }

In [53]:
GCV =  GridSearchCV(svc, grid_params, cv = 3, verbose=10)
GCV.fit(X_train,y_train)

Fitting 3 folds for each of 8 candidates, totalling 24 fits
[CV 1/3; 1/8] START dual=True, intercept_scaling=2, loss=hinge, multi_class=ovr, penalty=l2
[CV 1/3; 1/8] END dual=True, intercept_scaling=2, loss=hinge, multi_class=ovr, penalty=l2;, score=0.931 total time=   1.2s
[CV 2/3; 1/8] START dual=True, intercept_scaling=2, loss=hinge, multi_class=ovr, penalty=l2
[CV 2/3; 1/8] END dual=True, intercept_scaling=2, loss=hinge, multi_class=ovr, penalty=l2;, score=0.919 total time=   1.4s
[CV 3/3; 1/8] START dual=True, intercept_scaling=2, loss=hinge, multi_class=ovr, penalty=l2
[CV 3/3; 1/8] END dual=True, intercept_scaling=2, loss=hinge, multi_class=ovr, penalty=l2;, score=0.924 total time=   2.1s
[CV 1/3; 2/8] START dual=True, intercept_scaling=2, loss=squared_hinge, multi_class=ovr, penalty=l2
[CV 1/3; 2/8] END dual=True, intercept_scaling=2, loss=squared_hinge, multi_class=ovr, penalty=l2;, score=0.924 total time=   1.8s
[CV 2/3; 2/8] START dual=True, intercept_scaling=2, loss=squared

GridSearchCV(cv=3, estimator=LinearSVC(),
             param_grid={'dual': [True, False], 'intercept_scaling': [2, 3],
                         'loss': ['hinge', 'squared_hinge'],
                         'multi_class': ['ovr'], 'penalty': ['l2']},
             verbose=10)

In [54]:
## Printing the best hyper parameter values
GCV.best_params_   

{'dual': True,
 'intercept_scaling': 2,
 'loss': 'hinge',
 'multi_class': 'ovr',
 'penalty': 'l2'}

In [55]:
#training and testing our final model with above parameters
model = LinearSVC(dual = True, intercept_scaling = 2, loss = 'hinge', multi_class = 'ovr', penalty = 'l2')
model.fit(X_train,y_train) #fitting data to model
pred = model.predict(X_test)
accuracy = accuracy_score(y_test,pred)*100

#printing accuracy score
print("Accuracy Score :", accuracy)

#printing Confusion matrix
print(f"\nConfusion Matrix : \n {confusion_matrix(y_test,pred)}\n")

#printing Classification report
print(f"\nCLASSIFICATION REPORT : \n {classification_report(y_test,pred)}")

Accuracy Score : 92.93680297397769

Confusion Matrix : 
 [[ 32   0   0   0   9]
 [  0   3   0   3   0]
 [  0   0  29   9  10]
 [  0   0   0 141  23]
 [  0   0   0   3 545]]


CLASSIFICATION REPORT : 
               precision    recall  f1-score   support

           1       1.00      0.78      0.88        41
           2       1.00      0.50      0.67         6
           3       1.00      0.60      0.75        48
           4       0.90      0.86      0.88       164
           5       0.93      0.99      0.96       548

    accuracy                           0.93       807
   macro avg       0.97      0.75      0.83       807
weighted avg       0.93      0.93      0.93       807



In [56]:
## SAving the model
import joblib
joblib.dump(model,"Rating_Prediction_Project.pkl")

['Rating_Prediction_Project.pkl']